import

In [1]:
%matplotlib inline
import os
import torch

import matplotlib.pyplot as plt
import torchvision.models as models

# from flashtorch.utils import apply_transforms, load_image
# from flashtorch.saliency import Backprop

import dataLoader
from configParser import ConfigParser, getModelName

cuda

In [2]:
# CUDA support 
if torch.cuda.is_available():
    torch.cuda.set_device(0) # 0
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
    print("We are using cuda")

We are using cuda


parameters

In [3]:
# params
experimentName = "test_22987_suffix50_11_CNN2_differentConfig_batchnorm"
experiment_index = 0
trial_index=2
experiment_type = "multi-trial"

importLoad experiment

In [4]:
from configParser import ConfigParser
config_parser = ConfigParser(experimentName)

experimentName = os.path.join(experimentName, experiment_type)
print(experimentName)

test_22987_suffix50_11_CNN2_differentConfig_batchnorm/multi-trial


load dataset

In [5]:
if experiment_type == "multi-trial" or experiment_type == "selected-multi-trial":
    if experiment_type == "multi-trial":
        paramsIterator = config_parser.getHyperpIter()
    else:
        paramsIterator = config_parser.getHyperpSelectedIter()
        
    for indx, e in enumerate(paramsIterator):
        if indx ==experiment_index:
            experiment_params = e
            break
            
elif experiment_type == "single-trial":
    experiment_params = config_parser.read()
else:
    raise


print(experiment_params)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 4000, 'learning_rate': 0.01, 'numOfTrials': 10, 'patience': 100, 'useHeirarchy': True, 'useRelu': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'fc_layers': 2, 'useAdam': False, 'resnet': '18', 'normalize': True, 'softmax': True, 'batchNormalize': True}


In [6]:
datasetManager = dataLoader.datasetManager(experimentName, True)
datasetManager.updateParams(experiment_params)
dataset = datasetManager.getDataset()

file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_CNN2_differentConfig_batchnorm/multi-trial/tc0.640000_vc0.160000_d224_c3_augFalse_nTrue/dataset.pkl read


Load model of a specific trial

In [7]:
import CNN
import pandas as pd

speciesList = dataset.getSpeciesList()
numberOfSpecies = len(speciesList)
genusList = dataset.getGenusList()
numberOfGenus = len(genusList)

trialName = experimentName
if experiment_type != "single-trial":
    trialName = os.path.join(experimentName, getModelName(experiment_params, trial_index))

architecture = {
    "species": numberOfSpecies,
    "genus" : numberOfGenus
}
model = CNN.create_model(architecture, experiment_params)
loss_list, accuracy_list, epochs, time_elapsed = CNN.loadModel(model, trialName)

Top-k accuracy

In [8]:
def get_topk_dataframe(classType, lbllist, predlist):
    if classType=="genus":
        classes = range(numberOfGenus)
    else:
        classes = range(numberOfSpecies)

    df = pd.DataFrame(columns=[classType, 'count', 'top-1', 'top-2', 'top-3', 'top-4', 'top-5'])

    for class_ in classes:
        lbllist_idxs = (lbllist==class_).nonzero()[:,0]
        lbllist_sub = lbllist[lbllist_idxs, :]
        predlist_sub = predlist[lbllist_idxs, :]

        tops = CNN.top_k_acc(predlist_sub, lbllist_sub)
        df = df.append({classType : class_ , 
                        'count' : lbllist_sub.shape[0],
                        'top-1' : tops[0].cpu().numpy(), 
                        'top-2' : tops[1].cpu().numpy(),
                        'top-3' : tops[2].cpu().numpy(),
                        'top-4' : tops[3].cpu().numpy(),
                        'top-5' : tops[4].cpu().numpy(),
                        } , ignore_index=True)


    tops = CNN.top_k_acc(predlist, lbllist)
    df = df.append({classType : "Overall" , 
                    'count' : predlist.shape[0],
                    'top-1' : tops[0].cpu().numpy(), 
                    'top-2' : tops[1].cpu().numpy(),
                    'top-3' : tops[2].cpu().numpy(),
                    'top-4' : tops[3].cpu().numpy(),
                    'top-5' : tops[4].cpu().numpy(),
                    } , ignore_index=True)
    
    return df

In [9]:
def get_topk_withinRightGenus_dataframe(lbllist, predlist, dataset):
    df = pd.DataFrame(columns=['species', 'count', 'top-1', 'top-2', 'top-3', 'top-4', 'top-5'])

    numberOfSpecies = len(dataset.getSpeciesList())
    for class_ in range(numberOfSpecies):
        lbllist_idxs = (lbllist==class_).nonzero()[:,0]
        lbllist_sub = lbllist[lbllist_idxs, :]
        predlist_sub = predlist[lbllist_idxs, :]

        tops = CNN.top_k_acc_within_genus(predlist_sub, lbllist_sub, dataset)
        df = df.append({'species' : class_ , 
                        'count' : predlist_sub.shape[0],
                        'top-1' : tops[0].cpu().numpy(), 
                        'top-2' : tops[1].cpu().numpy(),
                        'top-3' : tops[2].cpu().numpy(),
                        'top-4' : tops[3].cpu().numpy(),
                        'top-5' : tops[4].cpu().numpy(),
                        } , ignore_index=True)

    tops = CNN.top_k_acc_within_genus(predlist, lbllist, dataset)
    df = df.append({'species' : "Overall" , 
                    'count' : predlist.shape[0],
                    'top-1' : tops[0].cpu().numpy(), 
                    'top-2' : tops[1].cpu().numpy(),
                    'top-3' : tops[2].cpu().numpy(),
                    'top-4' : tops[3].cpu().numpy(),
                    'top-5' : tops[4].cpu().numpy(),
                    } , ignore_index=True)

    return df

In [10]:
from IPython.display import display, HTML

train_loader, validation_loader, test_loader = datasetManager.getLoaders()

predlist_species, lbllist_species = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, label="species")
df_Species = get_topk_dataframe("species", lbllist_species, predlist_species)

predlist_genus, lbllist_genus = CNN.getLoaderPredictionProbabilities(test_loader, model, experiment_params, label="genus")
df_Genus = get_topk_dataframe("genus", lbllist_genus, predlist_genus)

print("Top-5 accuracy by Genus and Species")
display(HTML(df_Species.to_html()))
display(HTML(df_Genus.to_html()))

Loading saved dataloaders...
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_CNN2_differentConfig_batchnorm/multi-trial/tc0.640000_vc0.160000_d224_c3_augFalse_nTrue/trainingLoader.pkl read
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_CNN2_differentConfig_batchnorm/multi-trial/tc0.640000_vc0.160000_d224_c3_augFalse_nTrue/valLoader.pkl read
file /data/BGNN_data/INHS_cropped/50_11/test_22987_suffix50_11_CNN2_differentConfig_batchnorm/multi-trial/tc0.640000_vc0.160000_d224_c3_augFalse_nTrue/testLoader.pkl read
Top-5 accuracy by Genus and Species


,species,count,top-1,top-2,top-3,top-4,top-5
0,0,10,80.0,90.0,90.0,90.0,100.0
1,1,10,70.0,100.0,100.0,100.0,100.0
2,2,10,80.0,100.0,100.0,100.0,100.0
3,3,10,80.0,90.0,100.0,100.0,100.0
4,4,10,80.0,80.0,100.0,100.0,100.0
5,5,10,100.0,100.0,100.0,100.0,100.0
6,6,10,70.0,90.0,100.0,100.0,100.0
7,7,10,90.0,100.0,100.0,100.0,100.0
8,8,10,40.0,80.0,80.0,80.0,80.0
9,9,10,100.0,100.0,100.0,100.0,100.0


,genus,count,top-1,top-2,top-3,top-4,top-5
0,0,20,95.0,95.0,95.0,100.0,100.0
1,1,10,90.0,100.0,100.0,100.0,100.0
2,2,10,80.0,90.0,100.0,100.0,100.0
3,3,20,95.0,100.0,100.0,100.0,100.0
4,4,50,100.0,100.0,100.0,100.0,100.0
5,Overall,110,95.45455,98.18182,99.09091,100.0,100.0


In [11]:
df_Species_within_correct_genis = get_topk_withinRightGenus_dataframe(lbllist_species, predlist_species, dataset)

print("Top-5 accuracy by Species being within correct Genus")
display(HTML(df_Species_within_correct_genis.to_html()))

Top-5 accuracy by Species being within correct Genus


,species,count,top-1,top-2,top-3,top-4,top-5
0,0,10,90.0,90.0,90.0,90.0,100.0
1,1,10,100.0,100.0,100.0,100.0,100.0
2,2,10,80.0,100.0,100.0,100.0,100.0
3,3,10,80.0,90.0,100.0,100.0,100.0
4,4,10,90.0,90.0,100.0,100.0,100.0
5,5,10,100.0,100.0,100.0,100.0,100.0
6,6,10,100.0,100.0,100.0,100.0,100.0
7,7,10,100.0,100.0,100.0,100.0,100.0
8,8,10,100.0,100.0,100.0,100.0,100.0
9,9,10,100.0,100.0,100.0,100.0,100.0
